<a href="https://colab.research.google.com/github/churry75/tf_tutorial/blob/master/tf_tutorial_04_Subclassing_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SubClassing API

## Sequential API와 Funtional API의 특징
- 두 함수 모두 선언적(declarative)이다.
    - 사용할 층과 연결 방식을 먼저 정의해야 됨
    - 이 다음, 모델에 데이터를 주입하여 훈련이나 추론을 함
1. 장점
    - 모델의 저장, 복사, 공유가 쉽다.
    - 모델의 구조를 출력하거나 분석하기 용이하다.
    - 모델에 데이터가 주입되기 전에, 프레임워크가 크기를 짐작하고 타입을 확인하여 에러를 일찍 발견할 수 있다.(?)
    - **정적 그래프**(모델이 층으로 구성된)이므로 디버깅하기 쉽다.

## Subclassing API 사용 이유
- 위의 두 API의 정적 스타일을 보완하기 위해 **동적인 구조**(반복문, 조건문 등)로 프로그래밍하기 위해 사용

## Subclassing API 구현
- Model에 관한 Class를 상속한 다음 생성자 안에서 필요층을 만든다.
- call() 메소드 안에 수행하려는 연산을 기술

In [1]:
import tensorflow as tf
from tensorflow import keras

In [5]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

In [6]:
model = WideAndDeepModel()

- 위와 같이 클래스로 만들면 유연성이 높아짐(for, if, 저수준 텐서플로우 연산 사용 가능)
- 그러나 유연성이 높아진 만큼 이에 따른 비용이 발생하게 됨
    - 모델 구조가 call 메서드 안에 숨어져 있기 때문에, keras가 이를 분석하지 못 함
        - model.summary() 메서드로 모델 정보 확인 불가능
    - keras가 타입과 크기를 미리 확인할 수 없어 실수가 발생하기 쉽다.(?)
    - 따라서, 유연성이 크게 필요하지 않다면 sequential API 이나 Functional API를 사용하길 추천함